In [28]:
import csv
import cv2
import numpy as np

lines = []
with open('./data/lap1/driving_log.csv') as csvfile:
  reader = csv.reader(csvfile)
  for line in reader:
    lines.append(line)
    
images = []
measurements = [] 
for line in lines:
  for i in range(3):
    source_path = line[0]
    tokens = source_path.split('/')
    filename = tokens[-1]  
    local_path = "./data/lap1/IMG/" + filename
    image = cv2.imread(local_path)
    images.append(image)
  correction = 0.2
  measurement = float(line[3])
  measurements.append(measurement)
  measurements.append(measurement + correction)
  measurements.append(measurement - correction)

    
augmented_images = []
augmented_measurements = []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    flipped_image = cv2.flip(image, 1)
    flipped_measurement = float(measurement) * 1.0
    augmented_images.append(flipped_image)
    augmented_measurements.append(flipped_measurement)
    
    
    
X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

In [29]:
print(X_train.shape)

(12726, 160, 320, 3)


In [30]:
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

#lenet implemented

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(6, 5, 5, activation='relu'))
model.add(MaxPooling2D())
model.add(Convolution2D(16, 5, 5, activation='relu'))
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=3)

model.save('model.h5')


/Users/melihtt/tools/miniconda3/envs/cenv-p36-car/lib/python3.5/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
/Users/melihtt/tools/miniconda3/envs/cenv-p36-car/lib/python3.5/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (5, 5), activation="relu")`
  del sys.path[0]
/Users/melihtt/tools/miniconda3/envs/cenv-p36-car/lib/python3.5/site-packages/keras/models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 10180 samples, validate on 2546 samples
Epoch 1/5
10180/10180 [==============================] - 455s 45ms/step - loss: 28.0452 - val_loss: 0.0536
Epoch 2/5
10180/10180 [==============================] - 433s 43ms/step - loss: 0.0606 - val_loss: 0.0526
Epoch 3/5
10180/10180 [==============================] - 1514s 149ms/step - loss: 0.0556 - val_loss: 0.0524
Epoch 4/5
10180/10180 [==============================] - 425s 42ms/step - loss: 0.0508 - val_loss: 0.0528
Epoch 5/5
10180/10180 [==============================] - 426s 42ms/step - loss: 0.0462 - val_loss: 0.0524
